In [8]:
#import re  # Для препроцессинга
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [9]:
# Загружаю данные
df_clean = pd.read_csv(r'C:\Users\Daniel\project university\Выпускная работа\PRE.csv')

In [10]:
df_clean.head(-5)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр отдельный л...
2,первомайский мина пешкомт плая уютный квартира...
3,шаговый доступность парк сокольники развитой и...
4,ильинский коммуникация центральный квартира хо...
...,...
100713,переуступка входить цена корпус комната изолир...
100714,продаваться трехкомнатная квартира замечательн...
100715,прочитайте объявление полностью звонить строго...
100716,читать звоним согласно статья фз исполнительны...


In [11]:
#использую реализацию Gensim Doc2Vec
import multiprocessing
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [12]:
cores = multiprocessing.cpu_count() # Количество ядер на компе

In [13]:
df_clean['clean']=df_clean['clean'].fillna("Без описания")# меняю нан на "Без описания"

In [14]:
#токенизация и добавление в текст
description = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df_clean.clean)]

In [15]:
#размер вектора, количество измерений, количество переборов, количество ядер
d2v_model = Doc2Vec(vector_size=64, min_count=1, epochs = 20, workers=cores)

INFO - 15:47:47: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w5,s0.001,t16)', 'datetime': '2022-11-27T15:47:47.657674', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [16]:
#Модель
d2v_model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=cores, epochs = 30)
#Словарь
d2v_model.build_vocab(description)
#Обучение модели
d2v_model.train(description, total_examples=d2v_model.corpus_count
            , epochs=d2v_model.epochs)

INFO - 15:47:50: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w2,s0.001,t16)', 'datetime': '2022-11-27T15:47:50.735651', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 15:47:50: collecting all words and their counts
INFO - 15:47:50: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 15:47:50: PROGRESS: at example #10000, processed 633411 words (7642101/s), 14572 word types, 0 tags
INFO - 15:47:50: PROGRESS: at example #20000, processed 1224343 words (7281881/s), 19483 word types, 0 tags
INFO - 15:47:50: PROGRESS: at example #30000, processed 1776316 words (7143949/s), 21530 word types, 0 tags
INFO - 15:47:51: PROGRESS: at example #40000, processed 2331132 words (7099173/s), 23159 word types, 0 tags
INFO - 15:47:51: PROGRESS: at example #50000, processed 3260837 words (7689683/s), 25968 word types, 0 tags
INFO - 15:47:51: PROGRESS: a

In [33]:
#Генерация векторов
des2vec = [d2v_model.infer_vector((df_clean['clean'][i].split(' '))) for i in range(0,len(df_clean['clean']))]
des2vec

[array([ 0.23170717, -0.01758228, -0.00547968, -0.0582979 , -0.0541252 ,
        -0.16704695,  0.5551476 , -0.52133626, -0.00142984,  0.03499619,
        -0.07836372,  0.22946405, -0.00754282,  0.59211606,  0.40374565,
         0.18801805,  0.32137907, -0.03613994,  0.266108  , -0.32221088,
         0.13551246, -0.21525705, -0.15635219, -0.21177183,  0.433318  ,
         0.09784552, -0.2263196 , -0.07735508,  0.02656811,  0.21675347,
        -0.36817887, -0.10150539, -0.46033502, -0.09075385,  0.01228694,
         0.11943554, -0.2543223 , -0.3931368 ,  0.59408194,  0.14230251,
        -0.1830933 ,  0.4191546 ,  0.5500247 , -0.02380134, -0.5094126 ,
         0.47063437,  0.42398837, -0.45265713,  0.3194058 ,  0.5308391 ,
        -0.19027485,  0.39316308, -0.14166352, -0.23245975, -0.0373074 ,
         0.03537742, -0.10347956, -0.09436808,  0.00699187,  0.12938258,
        -0.19554344, -0.0619134 , -0.6708668 ,  0.50838196], dtype=float32),
 array([ 0.6439442 ,  0.27114442,  0.14381355, 

In [136]:
des2vec[0].dtype


dtype('float32')

In [137]:
from statistics import mean 
vec=[]
for i in range (0,len(des2vec),1):
    vec1 = des2vec[i] 
    vec_mean= mean(vec1)
    vec.append(vec_mean)
    i=i+1

vec

[0.026038213,
 -0.016835045,
 -0.016803036,
 -0.0027547101,
 -0.0075797285,
 -0.012284122,
 -0.012499596,
 0.07859775,
 0.0035330637,
 -0.000974902,
 -0.043811664,
 0.0416759,
 0.038303994,
 0.0016905623,
 0.07410315,
 -0.017631557,
 0.0027164987,
 0.03141549,
 0.012630833,
 -0.0034894114,
 -0.023858076,
 -0.0026630491,
 -0.024175027,
 -0.025012076,
 -0.009681666,
 -0.05044831,
 -0.035357557,
 0.16220063,
 -0.06253308,
 0.0037940156,
 -0.053719427,
 -0.028043233,
 0.117871225,
 -0.0065258606,
 0.0066703986,
 -0.024985932,
 -0.012557941,
 0.05712252,
 0.018753355,
 0.01187281,
 0.017177464,
 0.013087898,
 0.01452619,
 0.081763506,
 -0.039746515,
 -0.0006211464,
 -0.054134626,
 0.05651563,
 -0.022443278,
 -0.041851778,
 -0.14264575,
 0.08722896,
 -0.023350948,
 0.011744812,
 0.019605141,
 -0.025962021,
 0.026336769,
 0.055050366,
 0.013760804,
 -0.0171873,
 0.049487907,
 -0.051965196,
 -0.0017152588,
 0.009280698,
 0.040194348,
 -0.02125641,
 -0.04502384,
 -0.002236685,
 -0.004906004,
 0

In [138]:
import numpy as np
#Создание списка списков
#добавление списка в столбец данных
df_clean['doc2vec'] = np.array(vec).tolist()

#Это не обязательное условие
#df_clean= df_clean.drop([0],axis=1)

df_clean.head(-5)

,clean,doc2vec
0,прекрасный однокомнатный квартира дом лифт мус...,0.026038
1,светлый просторный комната кв метр отдельный л...,-0.016835
2,первомайский мина пешкомт плая уютный квартира...,-0.016803
3,шаговый доступность парк сокольники развитой и...,-0.002755
4,ильинский коммуникация центральный квартира хо...,-0.007580
...,...,...
100713,переуступка входить цена корпус комната изолир...,-0.000006
100714,продаваться трехкомнатная квартира замечательн...,-0.053091
100715,прочитайте объявление полностью звонить строго...,-0.000494
100716,читать звоним согласно статья фз исполнительны...,0.011123


In [141]:
df_clean.to_csv(r'C:\Users\Daniel\project university\Выпускная работа\2vec.csv',index=False)

In [144]:
df1 = pd.read_csv(r'C:\Users\Daniel\project university\Выпускная работа\Сlean.csv')
df2 = pd.read_csv(r'C:\Users\Daniel\project university\Выпускная работа\2vec.csv')

In [149]:
Data = pd.concat([df1, df2], axis=1)
Data.head(2)

,id,rooms,cost(₽),square(м²),cost_for_meter(₽),address,metro_name,distance(м),center_distance/m,floor,floors,description,clean,doc2vec
0,0,1 – комнатная,5000000.0,31.0,161290.3,"Центральная ул., 6",бульвар дмитрия донского,4995.0,20600.0,2,4,Прекрасная однокомнатная квартира в доме с лиф...,прекрасный однокомнатный квартира дом лифт мус...,0.026038
1,1,1 комната,4500000.0,98.0,45918.4,"Коптевская ул., 28к2",коптево,420.0,11400.0,3,5,"Светлая, просторная комната 19 кв. метров. Отд...",светлый просторный комната кв метр отдельный л...,-0.016835


In [150]:
Data=Data.drop(['clean'],axis=1)
Data.head(2)

,id,rooms,cost(₽),square(м²),cost_for_meter(₽),address,metro_name,distance(м),center_distance/m,floor,floors,description,doc2vec
0,0,1 – комнатная,5000000.0,31.0,161290.3,"Центральная ул., 6",бульвар дмитрия донского,4995.0,20600.0,2,4,Прекрасная однокомнатная квартира в доме с лиф...,0.026038
1,1,1 комната,4500000.0,98.0,45918.4,"Коптевская ул., 28к2",коптево,420.0,11400.0,3,5,"Светлая, просторная комната 19 кв. метров. Отд...",-0.016835
